In [1]:
# start
import time
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
# Crear Dataframe con los nombres de las columnas
coords = ["x", "y", "z"]    # opcional para nombrar puntos
list_column_names = []  # Lista de nombres de columnas
# Palabra o clase columna
list_column_names.append("palabra")
list_column_names.append("IDv") # para identificar los frames de un video en dataframe
# MANO DERECHA crear nombre de columnas 
count = 0   # contador de puntos
for landmark in mp_hands.HandLandmark:
    for coord in coords:
        list_column_names.append("Derecha"+"_" + str(count) + "_" + coord)
    count+=1

# MANO IZQUIERDA crear nombre de columas
count = 0   # contador de puntos
for landmark in mp_hands.HandLandmark:
    for coord in coords:
        list_column_names.append("Izquierda"+"_" + str(count) + "_" + coord)
    count+=1

In [4]:
#CREAR DATAFRAME CON EL RESULTADO   21 * 2 * 3 = 126 puntos mas clase 127
Dataframe = pd.DataFrame(columns = [list_column_names])
Dataframe

,palabra,IDv,Derecha_0_x,Derecha_0_y,Derecha_0_z,Derecha_1_x,Derecha_1_y,Derecha_1_z,Derecha_2_x,Derecha_2_y,...,Izquierda_17_z,Izquierda_18_x,Izquierda_18_y,Izquierda_18_z,Izquierda_19_x,Izquierda_19_y,Izquierda_19_z,Izquierda_20_x,Izquierda_20_y,Izquierda_20_z


In [5]:
# uno = np.random.rand(127)
# uno.shape
# ==================Para covertir un array de numpy a dataframe
# unoo = pd.DataFrame(uno)

# ==================Para agregar uno fila a un dataframe
# Dataframe.loc[0] = uno

In [6]:
len(Dataframe)  # Longitud del dataframe

0

In [7]:
# leer videos
path = "../Modelos/RegresionLinealClasificacionDinamic/DatasetGestoDinamicoHD/" # Ruta de videos
words = [words for words in os.listdir(path) if os.path.isdir(os.path.join(path, words))]
# Todos los videos se cambian a  resolucion (600x900), para su mejor deteccion de puntos.
print(words)
IDVIDEO = 0;
WIDTH  = 620
HEIGHT = 346
carpetas = 0
for word in words:  # Nombre de directorios
    for video in os.listdir(os.path.join(path, word)):
        # Direccion donde se encuentran los videos
        video_path = os.path.join(os.path.join(path, word), video)
        print(video_path)   # Nombre del video
        cap = cv2.VideoCapture(video_path) # Captura de videos uno por uno
        contadorFrames = 0  # contador de frames con puntos encontrados
        # Inizializar modelo holistico
        with mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            model_complexity=1,
            min_detection_confidence=0.5) as hands:
            while cap.isOpened():
                # Leer sequencia
                ret, frame = cap.read()
                if ret == True:
                    # Redimensionar sequencia
                    # frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation = cv2.INTER_AREA)
                    # ==========================    Se deberia voltear los frames
                    frame = cv2.flip(frame, 1)
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Cambiar  color de BGR a RGB
                    frame.flags.writeable = False
                    # Le pasamos al model para deteccion de puntos de referencia
                    results = hands.process(frame)
                    image_height, image_width, _ = frame.shape
                    # Si se encuentran puntos incompletos que no sean 21 0 42 no guarda
                    try:
                        contadorFrames+=1   # Si hay puntos en el frame incrementa
                        row = [] # Añadir coordenadas a la lista
                        # El nombre de las palabras es añadida a la lista "clase"
                        row.append(word)
                        row.append(IDVIDEO)
                        # print(results.multi_handedness)
                        print("frames: ", contadorFrames)   # frames en las que se encuentra mano/s
                        # print(results.multi_hand_landmarks)
                        for j, landmark in enumerate(results.multi_hand_landmarks):  # return 21 o 42 points 1 fps
                            # print(j)
                            for (i, points) in enumerate(landmark.landmark):
                                # print(i, points)  # indice y el punto en sus 3 coordenadas
                                row.append(points.x)
                                row.append(points.y)
                                row.append(points.z)
                        # print(len(row))
                        if(len(row) == 128): # si se encuentran las dos manos
                            Dataframe.loc[len(Dataframe)] = row
                            row = []
                        else:   # Llenar de ceros si se detecto una sola mano
                            rowZeros = np.zeros(63)
                            rowComplete = np.append(row, rowZeros, axis=0)
                            Dataframe.loc[len(Dataframe)] = rowComplete
                            row = []
                    except:
                        print("No se detecto ninguna mano")
                        row = []
                        pass
                else:
                    break
            cap.release()
            IDVIDEO+=1
        print("="*50,"  :  ",carpetas,"="*50)
    carpetas+=1

['Hola', 'BuenasTardes', 'BuenosDias', 'Gracias', 'Chau', 'Puedo', 'BuenasNoches', 'Permiso', 'Saludos', 'PorFavor']
../Modelos/RegresionLinealClasificacionDinamic/DatasetGestoDinamicoHD/Hola/word_hola_003.mp4
frames:  1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


frames:  2
frames:  3
frames:  4
frames:  5
frames:  6
frames:  7
frames:  8
frames:  9
==================================================   :   0 ==================================================
../Modelos/RegresionLinealClasificacionDinamic/DatasetGestoDinamicoHD/Hola/word_hola_013.mp4
frames:  1
frames:  2
frames:  3
frames:  4
frames:  5
frames:  6
frames:  7
frames:  8
frames:  9
frames:  10
frames:  11
==================================================   :   0 ==================================================
../Modelos/RegresionLinealClasificacionDinamic/DatasetGestoDinamicoHD/Hola/word_hola_024.mp4
frames:  1
frames:  2
frames:  3
frames:  4
frames:  5
==================================================   :   0 ==================================================
../Modelos/RegresionLinealClasificacionDinamic/DatasetGestoDinamicoHD/Hola/word_hola_021.mp4
frames:  1
frames:  2
frames:  3
frames:  4
No se detecto ninguna mano
frames:  5
frames:  6
frames:  7
=====================

In [8]:
# rowZeros = np.zeros(63)
# rowComplete = np.append(row, rowZeros, axis=0)
# rowComplete.shape

In [9]:
Dataframe

,palabra,IDv,Derecha_0_x,Derecha_0_y,Derecha_0_z,Derecha_1_x,Derecha_1_y,Derecha_1_z,Derecha_2_x,Derecha_2_y,...,Izquierda_17_z,Izquierda_18_x,Izquierda_18_y,Izquierda_18_z,Izquierda_19_x,Izquierda_19_y,Izquierda_19_z,Izquierda_20_x,Izquierda_20_y,Izquierda_20_z
0,Hola,0,0.6059078574180603,0.3688112199306488,1.4052278629606008e-07,0.5824374556541443,0.3671073615550995,-0.002987705869600177,0.5691474676132202,0.3438445031642914,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hola,0,0.6079854965209961,0.3812544047832489,2.0556547042360762e-07,0.5822163224220276,0.37421298027038574,-0.0007258277619257569,0.5668532252311707,0.34743809700012207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Hola,0,0.6125495433807373,0.38873302936553955,1.701401117770729e-07,0.5869141221046448,0.3808964788913727,-0.001106075942516327,0.5694484710693359,0.353956937789917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Hola,0,0.6162694692611694,0.3897647559642792,1.6069859043454926e-07,0.5898366570472717,0.38254961371421814,-0.0006855587125755847,0.5712786912918091,0.3547746539115906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hola,0,0.6239961981773376,0.3862166106700897,1.2614580668923736e-07,0.5965043902397156,0.3799222707748413,-0.0011939776595681906,0.577092707157135,0.35060566663742065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4563,PorFavor,302,0.4190278947353363,0.6842875480651855,1.4464812636560964e-07,0.4367402493953705,0.6403360366821289,-0.004582210443913937,0.45973920822143555,0.6186429262161255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4564,PorFavor,302,0.40790995955467224,0.6585274338722229,1.5754838500470214e-07,0.42556777596473694,0.618242621421814,-0.0046664695255458355,0.44747859239578247,0.5938288569450378,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4565,PorFavor,302,0.40324270725250244,0.6360150575637817,1.4219399702142255e-07,0.4197547435760498,0.594429075717926,-0.004091838840395212,0.4411050081253052,0.5676044821739197,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4566,PorFavor,302,0.4056369662284851,0.6224803924560547,1.321479317084595e-07,0.4195935130119324,0.5787785053253174,-0.002542192116379738,0.44115734100341797,0.552076518535614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# GRABAR DATAFRAME EN ARCHIVO CSV
Dataframe.to_csv("./dataDinamico10ClassHD.csv", index= False)

In [13]:
data = pd.read_csv('./dataDinamico10ClassHD.csv')
data['palabra'].unique()

array(['Hola', 'BuenasTardes', 'BuenosDias', 'Gracias', 'Chau', 'Puedo',
       'BuenasNoches', 'Permiso', 'Saludos', 'PorFavor'], dtype=object)

In [ ]:
def static_img():
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
            
        image = cv2.imread('./manos001.jpg')  # Insert your Image Here
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_hand_landmarks:
            print("Continue")

        # Print handedness and draw hand landmarks on the image.
        print('Handedness:', results.multi_handedness)
        # Captura el ancho y alto de la imagen
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
            print('hand_landmarks:', hand_landmarks)
            print(
                f'Index finger tip coordinates: (',
                f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
            )
            mp_drawing.draw_landmarks(
                annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imwrite(r'hands.png', annotated_image)

static_img()

In [5]:
# For webcam input:
cap = cv2.VideoCapture(0)
def webcam():
    prevTime = 0
    with mp_hands.Hands(
            min_detection_confidence=0.5,  # Detection Sensitivity
            min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # Flip the image horizontally for a later selfie-view display, and convert
            # image = cv2.flip(image, 1)
            # the BGR image to RGB.
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)
            # Print handedness and draw hand landmarks on the image.
            # print('Handedness:', results.multi_handedness)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            image_height, image_width, _ = image.shape

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # =============================================================
                    print(
                        f'Index finger tip coordinates: (',
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height}, '
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z}) '
                    )
                    # =============================================================
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            currTime = time.time()
            fps = 1 / (currTime - prevTime)
            prevTime = currTime
            cv2.putText(image, f'FPS: {int(fps)}', (20, 70),
                        cv2.FONT_HERSHEY_PLAIN, 3, (0, 196, 255), 2)
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()


# choose webcam or static_img
webcam()

# end

Index finger tip coordinates: ( 834.1084289550781, 296.4125061035156, -0.04126456007361412) 
Index finger tip coordinates: ( 841.6394805908203, 299.94632720947266, -0.03897540643811226) 
Index finger tip coordinates: ( 842.0165252685547, 316.6783547401428, -0.04282555356621742) 
Index finger tip coordinates: ( 1029.328384399414, 214.24713134765625, -0.07347191870212555) 
Index finger tip coordinates: ( 1006.0720825195312, 212.13998794555664, -0.05430245026946068) 
Index finger tip coordinates: ( 986.0660552978516, 207.62173175811768, -0.04057587683200836) 
Index finger tip coordinates: ( 974.3247222900391, 202.0207643508911, -0.03925469145178795) 
Index finger tip coordinates: ( 970.1954650878906, 199.3805480003357, -0.03879428654909134) 
Index finger tip coordinates: ( 965.4961395263672, 196.99512004852295, -0.035443276166915894) 
Index finger tip coordinates: ( 962.5899505615234, 192.46270179748535, -0.0369800329208374) 
Index finger tip coordinates: ( 969.0412902832031, 184.75087881

KeyboardInterrupt: 

In [7]:
# Detruye la ventana memergente abierta
cap.release()
cv2.destroyAllWindows()
